In [1]:
import numpy as np
import numpy.matlib

import matplotlib.pyplot as plt
import matplotlib.pylab as plt
from matplotlib.pyplot import figure

import scipy.integrate
from scipy.integrate import solve_ivp
from scipy.sparse import csr_matrix, triu, spdiags
from scipy.sparse.linalg import spsolve, splu
import scipy.sparse

# import sys
# np.set_printoptions(threshold=sys.maxsize)

import time
# import warnings
# warnings.filterwarnings('ignore')
# %matplotlib notebook
# %matplotlib widget

### Problem 1

### A1

In [2]:
# L = 10 
# x_change = np.linspace(-L, L, 200)
# deltax = x_change[-1] - x_change[-2]

In [3]:
L = 10
x = np.arange(-L, L, 0.1)
deltax = x[-1] - x[-2]

In [4]:
m = 200
zero = np.zeros((m))
one = np.ones((m))
Bin = np.array([-one, zero, one])
d = np.array([-1,0,1])
A = spdiags(Bin, d, m, m, format = 'csc')

In [5]:
A[0,-1] = 1
A[-1,0] = -1

/Users/annahuang/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [6]:
A = A/(2*deltax)

In [7]:
A1 = A.todense().copy()

In [8]:
# plt.spy(A)

### A2

In [9]:
f = lambda x: np.exp(-(x-5)**2)
xp = [-L, L]

dt = 0.5
term = 10 # t = 10 
x = np.arange(-L,L,0.1)
N = len(x)

x_evals = np.arange(0,term+dt, dt)

y0 = f(x)

In [10]:
# C = -0.5 
A_2 = (-0.5*A)

In [11]:
def PDE(t,x,A):
    u_t = A@x
    return u_t 

In [12]:
tic = time.time()
sol = solve_ivp(lambda t,x: PDE(t,x,A_2), xp, y0, t_eval = x_evals)
toc = time.time()
# print(toc-tic)

In [13]:
A2 = sol.y

In [14]:
# A2.shape

### A3

In [15]:
f = lambda x: np.exp(-(x-5)**2)
L = 10 
xp = [-L, L]

dt = 0.5
term = 10 # t = 10 

x_evals = np.arange(0,term+dt, dt)

y0 = f(x)

In [16]:
x_range = np.arange(-L,L,0.1)
# N = len(x)

In [17]:
# # define heaviside function 

# def heavi_func():
#     c = ([])

#     for x in x_range:
#         # c_array = -(1+2*np.sin(5*t)- np.heaviside((x-4),0))
#         c_array = np.heaviside((x-4),0)
#         c = np.append(c, c_array)
        
#     return c 

In [18]:
# Rewrite ux as A*u (do this calculation) find the vector that is equal to number of spacial points 
# c is a vector equal to number of spaital points 
# element wise multiplcation between c and Au 

def funcPDE(t,u,A):
    
    c = ([])
    
    for x in x_range:
        c_array = -(1+2*np.sin(5*t)- np.heaviside((x-4),0))
        # c_array = np.heaviside((x-4),0)
        c = np.append(c, c_array)
        
    u_t = np.multiply(c ,A@u) # element wise multipication 
    # u_t = np.multiply(-heavi_func(),A@u) # element wise multipication 
    return u_t


# def funcPDE(t,u,A):
#     c = -(1+2*np.sin(5*t)- np.heaviside((x-4),0))
#     A_3 = -(c*A)
#     u_t = A_3@u
#     return u_t

In [19]:
tic = time.time()
sol1 = solve_ivp(lambda t,u: funcPDE(t,u,A),xp, y0, t_eval = x_evals)
toc = time.time()
# print(toc-tic)

In [20]:
A3 = sol1.y

### Problem 2

### A4

In [21]:
# delta x or delta y 
n = 64
L = 10
l = np.linspace(-L, L, n+1)
dl = l[1]-l[0]

In [22]:
def A(m):

    # m = 64 # N value in x and y directions
    n = m*m # total size of matrix

    e1 = np.ones(n) # vector of ones
    Low1 = np.tile(np.concatenate((np.ones(m-1), [0])), (m,)) # Lower diagonal 1
    Low2 = np.tile(np.concatenate(([1], np.zeros(m-1))), (m,)) #Lower diagonal 2
                                        # Low2 is NOT on the second lower diagonal,
                                        # it is just the next lower diagonal we see
                                        # in the matrix.

    Up1 = np.roll(Low1, 1) # Shift the array for spdiags
    Up2 = np.roll(Low2, m-1) # Shift the other array

    A = scipy.sparse.spdiags([e1, e1, Low2, Low1, -4*e1, Up1, Up2, e1, e1],
                             [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)], n, n,format = 'csr')
    
    return A

In [23]:
# plt.spy(A(4)) # view the matrix structure
# plt.show()

In [24]:
# A(64)[0,0] = 2
A = A(64)
A[0,0] = 2
A = A/(dl**2)

In [25]:
A4 = np.copy(A.toarray())

In [26]:
# print(A.toarray()[0:10,0:10])

### A5

In [27]:
def B(m):
    # m = 5
    n = m*m
    e1_B = np.ones((n)).T # vector of ones

    d_B = np.array([-((m**2)-m), - m, m, (m**2)-m])
    Bin_B = np.array([e1_B, -e1_B, e1_B, -e1_B])


    B = spdiags(Bin_B,d_B, n, n, format = 'csc')
    B = B/(2 * dl)
    return B

In [28]:
B = B(64)

In [29]:
A5 = np.copy(B.toarray())

In [30]:
# plt.spy(B(4))
# plt.show()

In [31]:
# print(B.toarray()[64:66,0:2])

### A6

In [32]:
def C(m):
    # m = 5 
    n = m*m

    e1_C = np.ones(n) # vector of ones
    Low1_C = np.tile(np.concatenate((np.ones(m-1), [0])), (m,)) # Lower diagonal 1
    Low2_C = np.tile(np.concatenate(([1], np.zeros(m-1))), (m,)) #Lower diagonal 2

    Up1_C = np.roll(Low1_C, 1) 
    Up2_C = np.roll(Low2_C, m-1)

    C = scipy.sparse.spdiags([Low2_C, -Low1_C, Up1_C, -Up2_C],
                            [-m+1, -1, 1, m-1], n, n,format = 'csc')
    C = C/(2 * dl)
    return C
# plt.spy(C(5))
# plt.show()

In [33]:
C = C(64)

In [34]:
A6 = np.copy(C.toarray())

In [35]:
# print(C.toarray()[0:18,0:18])

### A7

In [36]:
v = 0.001
L = 10
xp = [0, 4]

# time
t_evals = np.linspace(0,4,9)
dt = 0.5

# delta x or delta y 
n = 64
l = np.linspace(-L, L, n+1)
dl = l[1]-l[0]

In [37]:
# # initial conditions 
# for x and y (1 - 64)

w0 = np.array([])

for x in range(1, 65):
    for y in range(1, 65):
        k = np.exp(-2*(x**2) - ((y**2)/20))
        w0 = np.append(w0,k)

In [38]:
def func_ivp(t, w):
    
    v = 0.001
    
    # update psi 
    psi = scipy.sparse.linalg.spsolve(A, w)
    
    dw = (v*A@w) - ((B@psi)*(C@w)) + ((C@psi)*(B@w))
    
    return dw 

In [39]:
sol = scipy.integrate.solve_ivp(lambda t, w: func_ivp(t, w), xp, w0, t_eval = t_evals)

In [40]:
A7 = sol.y.T

### A8

In [41]:
v = 0.001
L = 10
xp = [0, 4]

# time
t_evals = np.linspace(0,4,9)
dt = 0.5

# delta x or delta y 
n = 64
l = np.linspace(-L, L, n+1)
dl = l[1]-l[0]

In [42]:
# # initial conditions 
# for x and y (1 - 64)

w0 = np.array([])

for x in range(1, 65):
    for y in range(1, 65):
        k = np.exp(-2*(x**2) - ((y**2)/20))
        w0 = np.append(w0,k)

In [43]:
P,L,U = scipy.linalg.lu(A.todense())

def func_ivp_plu(t, w):
    
    v = 0.001
    
    # update psi 
    y = np.linalg.solve(L,P.T@w)
    psi = np.linalg.solve(U,y)
    
    dw = (v*A@w) - ((B@psi)*(C@w)) + ((C@psi)*(B@w))
    
    return dw 

In [44]:
sol2 = scipy.integrate.solve_ivp(lambda t, w: func_ivp(t, w), xp, w0, t_eval = t_evals)

In [45]:
A8 = sol2.y.T

### A9

In [46]:
q = sol2.y[:,0].reshape(64,64)

In [47]:
w = sol2.y[:,1].reshape(64, 64)

In [48]:
e = sol2.y[:,2].reshape(64, 64)

In [49]:
r = sol2.y[:,3].reshape(64, 64)

In [50]:
t = sol2.y[:,4].reshape(64, 64)

In [51]:
y = sol2.y[:,5].reshape(64, 64)

In [52]:
u = sol2.y[:,6].reshape(64, 64)

In [53]:
i = sol2.y[:,6].reshape(64, 64)

In [54]:
o = sol2.y[:,6].reshape(64, 64)

In [55]:
A9 = np.stack([q,w,e,r,t,y,u,i,o])

In [56]:
# stack = np.array([])

# for r in range(0,2):
#     d = sol1.y[:,r].reshape(64,64)
#     np.stack
    